In [1]:
from EduData import get_data
import os

dataset_dir = '2009_skill_builder_data_corrected'
dataset_filename = 'skill_builder_data_corrected.csv'
dataset_path = './' + dataset_dir + '/' + dataset_filename

if not os.path.exists(dataset_path):
    get_data('assistment-2009-2010-skill', './')

In [2]:
import random
import pandas as pd
import tqdm
import numpy as np

data = pd.read_csv(
    dataset_path,
    usecols=['order_id', 'user_id', 'sequence_id', 'skill_id', 'correct']
).fillna(0)

In [3]:
raw_question = data.skill_id.unique().tolist()
num_skill = len(raw_question)

def question_id_transfer(question):
    id2question = [p for p in raw_question]
    question2id = {}
    for i, p in enumerate(raw_question):
        question2id[p] = i

    return id2question, question2id


id2question, question2id = question_id_transfer(raw_question)

print("number of skills: %d" % num_skill)

number of skills: 124


In [4]:
def parse_all_seq(students):
    all_sequences = []
    for student_id in tqdm.tqdm(students, 'parse student sequence:\t'):
        student_sequence = parse_student_seq(data[data.user_id == student_id])
        all_sequences.extend(student_sequence)
    return all_sequences


def parse_student_seq(student):
    student = student.drop_duplicates(subset='order_id')
    sequence_ids = student.sequence_id.unique()
    sequences = []
    for seq_id in sequence_ids:
        seq = student[student.sequence_id == seq_id].sort_values('order_id')
        questions = [question2id[id] for id in seq.skill_id.tolist()]
        answers = seq.correct.tolist()
        sequences.append((questions, answers))
    return sequences


# [(question_sequence_0, answer_sequence_0), ..., (question_sequence_n, answer_sequence_n)]
sequences = parse_all_seq(data.user_id.unique())

parse student sequence:	: 100%|██████████| 4217/4217 [00:58<00:00, 71.94it/s] 


In [5]:
def train_test_split(data, train_size=.7, shuffle=True):
    if shuffle:
        random.shuffle(data)
    boundary = round(len(data) * train_size)
    return data[: boundary], data[boundary:]


train_sequences, test_sequences = train_test_split(sequences)

In [6]:
def sequences2tl(sequences, trgpath):
    with open(trgpath, 'a', encoding='utf8') as f:
        for seq in tqdm.tqdm(sequences, 'write into file: '):
            questions, answers = seq
            seq_len = len(questions)
            f.write(str(seq_len) + '\n')
            f.write(','.join([str(q) for q in questions]) + '\n')
            f.write(','.join([str(a) for a in answers]) + '\n')


# save triple line format for other tasks
train_data_path = './' + dataset_dir + '/train.txt'
test_data_path = './' + dataset_dir + '/test.txt'
if not os.path.exists(train_data_path):
    sequences2tl(train_sequences, train_data_path)
if not os.path.exists(test_data_path):
    sequences2tl(test_sequences, test_data_path)

In [7]:
# one-hot

MAX_STEP = 50
NUM_QUESTIONS = num_skill


def encode_one_hot(sequences, max_step, num_questions):
    question_sequences = np.array([])
    answer_sequences = np.array([])
    one_hot_result = []

    for questions, answers in tqdm.tqdm(sequences, 'convert to one-hot format: '):
        length = len(questions)
        # append questions' and answers' length to an integer multiple of max_step
        mod = 0 if length % max_step == 0 else (max_step - length % max_step)
        fill_content = np.zeros(mod) - 1
        questions = np.append(questions, fill_content)
        answers = np.append(answers, fill_content)
        # one hot
        q_seqs = questions.reshape([-1, max_step])
        a_seqs = answers.reshape([-1, max_step])
        for (i, q_seq) in enumerate(q_seqs):
            one_hot = np.zeros(shape = [max_step, 2 * num_questions])
            for j in range(max_step):
                index = int(q_seq[j] if a_seqs[i][j] > 0 else q_seq[j] + num_questions)
                one_hot[j][index] = 1
            one_hot_result = np.append(one_hot_result, one_hot)
    
    return one_hot_result.reshape(-1, max_step, 2 * num_questions)


train_one_hot_path = './' + dataset_dir + '/train_data.npy'
test_one_hot_path = './' + dataset_dir + '/test_data.npy'

# save one_hot data
if not os.path.exists(train_one_hot_path):
    train_data = encode_one_hot(train_sequences, MAX_STEP, NUM_QUESTIONS)
    np.save(train_one_hot_path, train_data)
if not os.path.exists(test_one_hot_path):
    test_data = encode_one_hot(test_sequences, MAX_STEP, NUM_QUESTIONS)
    np.save(test_one_hot_path, test_data)

convert to onehot format:   5%|▌         | 2172/41912 [02:43<49:55, 13.26it/s]  


KeyboardInterrupt: 